# Logistic Regression for QAM Demodulation in AWGN Channels

This code is provided as supplementary material of the lecture Machine Learning and Optimization in Communications (MLOC).<br>

This code serves as:
* an exercise to demodulate QAM symbols using multiclass logistic regression

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
    
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import PillowWriter # Disable if you don't want to save any GIFs.
from ipywidgets import interactive
import ipywidgets as widgets
%matplotlib inline 

# use pytorch device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("We are using the following device for learning:",device)

Here we use a simple AWGN model and normalize the constellations to unit energy

In [ ]:
constellations = {'16-QAM': np.array([-3,-3,-3,-3,-1,-1,-1,-1,1,1,1,1,3,3,3,3]) + 1j*np.array([-3,-1,1,3,-3,-1,1,3,-3,-1,1,3,-3,-1,1,3]), \
                  '16-APSK': np.array([1,-1,0,0,1.4,1.4,-1.4,-1.4,3,-3,0,0,5,-5,0,0]) + 1j*np.array([0,0,1,-1,1.4,-1.4,1.4,-1.4,0,0,4,-4,0,0,6,-6]), \
                  '4-test' : np.array([-1,2,0,4]) + 1j*np.array([0,0,3,0])}

# permute constellations so that it is visually more appealing with the chosen colormap
# also normalize constellation
for cname in constellations.keys():
    norm_factor = 1 / np.sqrt(np.mean(np.abs(constellations[cname])**2))
    constellations[cname] = constellations[cname][np.random.permutation(len(constellations[cname]))] * norm_factor
    

constellation = constellations['16-QAM']
n = len(constellation)

Simple AWGN channel

In [ ]:
def simulate_channel(x, SNR):  
    # noise variance per step    
    sigma = np.sqrt( 0.5 * (10**(-SNR/10.0)) )

    return x + sigma*(np.random.randn(len(x)) + 1j*np.random.randn(len(x)))    

In [ ]:
length = 10000

def plot_constellation(SNR):
    ti = np.random.randint(len(constellation),size=length)
    t = constellation[ti]
    r = simulate_channel(t, SNR)

    plt.figure(figsize=(6,6))
    font = {'size'   : 14}
    plt.rc('font', **font)
    plt.rc('text', usetex=True)
    plt.scatter(np.real(r), np.imag(r), c=ti, cmap='tab20')
    plt.xlabel(r'$\Re\{r\}$',fontsize=14)
    plt.ylabel(r'$\Im\{r\}$',fontsize=14)
    plt.axis('equal')
    plt.title('Received constellation (SNR = $%1.2f$\,dBm)' % (SNR))    
    
interactive_update = interactive(plot_constellation, SNR = widgets.FloatSlider(min=0.0,max=30.0,step=0.1,value=15, continuous_update=False, description='SNR (dB)', style={'description_width': 'initial'}, layout=widgets.Layout(width='50%')))


output = interactive_update.children[-1]
output.layout.height = '500px'
interactive_update

Carry out gradient descent. Please insert the relevant code for yourself.

In [ ]:
class logistic_regression_bias(nn.Module):
    def __init__(self, logits):
        # TODO: implement initialization
        
    def forward(self, x):
        # TODO implement forward mapping

Main program loop, carry out 1000 iterations of gradient descent for finding the weights

In [ ]:
# initialize model and copy to target device
model = logistic_regression_bias(len(constellation))
model.to(device)


# Negative Log-Likelihood Loss function, attention, requires LogSoftmax to compute logarithmic softmax outputs
# TODO implement loss function
loss_fn = 

# Adam Optimizer
# TODO specify optimizer
optimizer = 

# Build dataset
SNR = 19 #dB

# Transmit data (bits)
ti = np.random.randint(len(constellation),size=length)
t = constellation[ti]
# Channel output
r = simulate_channel(t, SNR)
inputs = np.column_stack((np.real(r), np.imag(r)))

# convert input vector to pytorch
# TODO fill out vectors
inputs_torch = 
labels_torch = 

n = len(constellation)


# main gradient descent loop
for i in range(10000):            
    outputs = model(inputs_torch)
    loss = loss_fn(outputs, labels_torch)
    
    # compute gradient
    # TODO compute gradient
    
    # optimize
    # TODO run optimizer
    
    # reset gradients
    # TODO reset gradient
    
    if i % 500 == 0:
        preds = np.argmax(model(inputs_torch).detach().cpu().numpy(), axis=1)        
        error_rate = np.mean(preds != ti)
        print('Step',i,' : ',error_rate,' error_rate with loss ',loss)

In [ ]:
ext_x = max(abs(np.real(r)))
ext_y = max(abs(np.imag(r)))
ext_max = max(ext_x,ext_y)*1.2

mgx,mgy = np.meshgrid(np.linspace(-ext_max,ext_max,200), np.linspace(-ext_max,ext_max,200))
meshgrid = torch.from_numpy( np.column_stack((np.reshape(mgx,(-1,1)),np.reshape(mgy,(-1,1)))) ).float().to(device)

decision_region = np.argmax(model(meshgrid).detach().cpu().numpy(), axis=1) / 16

plt.figure(figsize=(8,8))
#plt.contourf(mgx,mgy,decision_region.reshape(mgy.shape),cmap='tab20',vmin=0,vmax=1)
plt.scatter(meshgrid[:,0],meshgrid[:,1],c = decision_region,cmap='tab20', s=5, alpha=0.7)
plt.scatter(np.real(r), np.imag(r), c=ti, cmap='tab20')

plt.axis('scaled')
plt.xlim((-ext_max,+ext_max))
plt.ylim((-ext_max,+ext_max))
plt.xlabel(r'$\Re\{r\}$',fontsize=16)
plt.ylabel(r'$\Im\{r\}$',fontsize=16)
plt.title('Decision regision',fontsize=16) 

Try a more _Deep-Learning_ soft of doing things with mini-batches and directly using the cross-entropy loss function

In [ ]:
class logistic_regression_nosoftmax_bias(nn.Module):
    def __init__(self, logits):
        # TODO implement
                   
    def forward(self, x):
        # Linear function, first layer
        # TODO implement

In [ ]:
# initialize model
model = logistic_regression_nosoftmax_bias(len(constellation))
model.to(device)

# TODO select loss function
loss_fn_cn = 

# TODO select Adam optimizer
optimizer = 

# Build dataset
SNR = 19 #dB


dataset_length = np.linspace(100,length, 10000)

# main gradient descent loop
for i in range(10000):       
    
    # Transmit data (bits)
    ti = np.random.randint(len(constellation),size=int(dataset_length[i]))
    t = constellation[ti]
    # Channel output
    r = simulate_channel(t, SNR)
    
    # generate classifier input
    inputs = np.column_stack((np.real(r), np.imag(r)))

    # convert input vector to pytorch
    # TODO implement
    inputs_torch = 
    labels_torch = 

    n = len(constellation)


    outputs = model(inputs_torch)
    loss = loss_fn_cn(outputs, labels_torch)
    
    # compute gradient
    # TODO implement
    
    # optimize
    # TODO implement
    
    # reset gradients
    # TODO implement
    
    if i % 500 == 0:
        preds = np.argmax(model(inputs_torch).detach().cpu().numpy(), axis=1)        
        error_rate = np.mean(preds != ti)
        print('Step',i,' : ',error_rate,' error_rate with loss ',loss)

In [ ]:
ext_x = max(abs(np.real(r)))
ext_y = max(abs(np.imag(r)))
ext_max = max(ext_x,ext_y)*1.2

mgx,mgy = np.meshgrid(np.linspace(-ext_max,ext_max,200), np.linspace(-ext_max,ext_max,200))
meshgrid = torch.from_numpy( np.column_stack((np.reshape(mgx,(-1,1)),np.reshape(mgy,(-1,1)))) ).float().to(device)

decision_region = np.argmax(model(meshgrid).detach().cpu().numpy(), axis=1) / 16

plt.figure(figsize=(8,8))
#plt.contourf(mgx,mgy,decision_region.reshape(mgy.shape),cmap='tab20',vmin=0,vmax=1)
plt.scatter(meshgrid[:,0],meshgrid[:,1],c = decision_region,cmap='tab20', s=5, alpha=0.7)
plt.scatter(np.real(r), np.imag(r), c=ti, cmap='tab20')

plt.axis('scaled')
plt.xlim((-ext_max,+ext_max))
plt.ylim((-ext_max,+ext_max))
plt.xlabel(r'$\Re\{r\}$',fontsize=16)
plt.ylabel(r'$\Im\{r\}$',fontsize=16)
plt.title('Decision regision',fontsize=16)